# Lab 04. Dekoratory.

Dekoratory zostały opisane w dwóch dokumentach PEP:
* [PEP-0318](https://peps.python.org/pep-0318/) - wydany w 2003 roku opisuje dekoratory dla funkcji i metod (Python 2.4)
* [PEP-3129](https://peps.python.org/pep-3129/) - wydany w 2007 roku i rozszerza zastosowanie dekoratorów o dekoratory klas (Python 3.0)

Dekorator to funkcja, która otacza inną funkcję (ang. wraps lub wrapper).

In [19]:
def decorator(obj):
    return obj


def say_hello():
    print('Hello')


@decorator
def say_bla():
    print('Bla!')

In [22]:
# wywołanie dekoratora jest ekwiwalentem powniższego wywołania
# zapisujemy do zmiennej obiekt zwrócony przez dekorator, czyli tu funkcję
say_hello = decorator(say_hello)

In [24]:
# którą możemy następnie wywołać
say_hello()

Hello


In [21]:
say_bla()

Bla!


In [25]:
# zobaczmy inny przykład
# tutaj wywołanie funkcji udekorowanej wywołuje inną funkcję

def cos_innego():
    print("cos innego")


def decorator(obj):
    return cos_innego

@decorator
def main():
    print("hello")

main()

cos innego


In [29]:
# co tu się tak na prawdę dzieje?
# czym jest main? To jest alias na decorator, który zwraca obiekt funkcji cos_innego
main
# wywołanie main() wywołuje więc decorator(main), które z przekazanym main nie robi nic, ale jak zostało
# wspomniane zwraca cos_innego, a że ten main jest wywołany (przez () ) to wywołujemy faktycznie tylko
# funkcję cos_innego

<function __main__.cos_innego()>

Zobaczmy kolejny przykład, gdzie funkcja dekorowana jest wywoływana wewnątrz funkcji dekorującej.

In [34]:
def zrob_zrob(obj):
    obj()
    obj()


@zrob_zrob
def main():
    print("hello")

main

hello
hello


To jednak nie jest działanie, które znamy z praktyki. W praktyce opakowujemy wewnątrz dekoratora funkcję dekorującą jeszcze jedną funkcją.

In [35]:
def zrob_zrob(obj):
    def wrapper():
        obj()
        obj()
    return wrapper


@zrob_zrob
def main():
    print("hello")

main, main()

hello
hello


(<function __main__.zrob_zrob.<locals>.wrapper()>, None)

Taka konstrukcja również nie jest w pełni funkcjonalna. Obiekt zwracany jest tu typu `wrapper`, a nie naszej funkcji dekorowanej i jeżeli funkcja main coś zwraca, to ta informacja jest tracona. Nie ma tutaj również obsługi ewentualnych argumentów przekazanych do funkcji.

Dodajmy kilka z tych brakujących funkcjonalności.

In [44]:
# przekazujemy argumenty do wnętrza dekoratora

def zrob_zrob(obj):
    def wrapper(*args, **kwargs):
        obj(*args, **kwargs)
        obj(*args, **kwargs)
    return wrapper


@zrob_zrob
def main(x):
    print(f'{x}')

main, main(10)

10
10


(<function __main__.zrob_zrob.<locals>.wrapper(*args, **kwargs)>, None)

In [45]:
# jednak jeżeli nasza funkcja coś zwraca to ta informacja jest nadal tracona
# dodatkowo obiekt zwracany to funkcja wrapper

def zrob_zrob(obj):
    def wrapper(*args, **kwargs):
        obj(*args, **kwargs)
        obj(*args, **kwargs)
    return wrapper


@zrob_zrob
def main(x):
    return f'{x}'

main, main(10)

(<function __main__.zrob_zrob.<locals>.wrapper(*args, **kwargs)>, None)

In [68]:
#

def zrob_zrob(obj):
    def wrapper(*args, **kwargs):
        """to wrapper"""
        print("dekoruję...")
        res = obj(*args, **kwargs)
        return res
    return wrapper


@zrob_zrob
def main(x):
    """udekorowana"""
    return f'{x}'

main, main(10), main.__name__,  main.__doc__

dekoruję...


(<function __main__.zrob_zrob.<locals>.wrapper(*args, **kwargs)>,
 '10',
 'wrapper',
 'to wrapper')

In [69]:
# nadal obiekt zwracany jest typu wrapper, co może być problemem w przypadku kontroli typów
# wywołujemy return dla funkcji dekorowanej
# ale również dla wrapper

def zrob_zrob(obj):
    def wrapper(*args, **kwargs):
        print("dekoruję...")
        obj(*args, **kwargs)
        return obj(*args, **kwargs)
    wrapper.__name__ = obj.__name__
    wrapper.__doc__ = obj.__doc__
    return wrapper


@zrob_zrob
def main(x):
    """udekorowana"""
    return f'{x}'

main, main(10), main.__name__,  main.__doc__

dekoruję...


(<function __main__.zrob_zrob.<locals>.wrapper(*args, **kwargs)>,
 '10',
 'main',
 'udekorowana')

Aby jednak się z tym nie męczyć we własnych dekoratorach wykorzystamy dekorator stworzony do tego celu w module `functools`.

In [72]:
from functools import wraps

def zrob_zrob(obj):
    @wraps(obj)
    def wrapper(*args, **kwargs):
        print("dekoruję...")
        obj(*args, **kwargs)
        return obj(*args, **kwargs)
    return wrapper


@zrob_zrob
def main(x):
    """udekorowana"""
    return f'{x}'

main, main(10), main.__name__,  main.__doc__

dekoruję...


(<function __main__.main(x)>, '10', 'main', 'udekorowana')

To teraz czas na kilka dekoratorów, które mogą być faktycznie do czegoś wykorzystane.

In [83]:
from time import perf_counter


def execution_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = perf_counter()
        result = func(*args, **kwargs)
        stop = perf_counter()
        print(f"Funkcja wykonała się w {stop - start:.2f}s")
        return result
    return wrapper


@execution_time
def milion(val):
    [val for _ in range(1_000_000)]

milion(0)
milion('boom!')
milion(1/3)

Funkcja wykonała się w 0.06s
Funkcja wykonała się w 0.04s
Funkcja wykonała się w 0.04s


Dekoratory klasy nie różnią się sposobem ich deklaracji i wywołania i (za oficjalną dokumentacją) mogą wyglądać tak:
```python

class A:
  pass
# dekorowanie bez adnotacji
A = foo(bar(A))

# ekwiwalent
@foo
@bar
class A:
  pass
```

Skoro dekorator to po prostu funkcja, która opakowuje inną funkcję, to powinniśmy móc również przekazać argumenty do takiego dekoratora (nie mylić z funkcją dekorowaną, to już wiemy).

In [33]:
import time


def sleep(sleep_time):
    def inner(func):
        def wrapper(*args, **kwargs):
            print(f'Czekam {sleep_time} sekund...')
            time.sleep(sleep_time)
            return func(*args, **kwargs)
        return wrapper
    return inner
    
@sleep(sleep_time=5)
def wakeup():
    print("Czas minął!")

@sleep(sleep_time=2)
def wakeup_fast():
    print("Czas minął tak szybko!")

# bez argumentów nie zadziała w tym przypadku!
@sleep
def wakeup_how():
    print("Czas minął tak szybko!")

wakeup()
wakeup_fast()
wakeup_how()

Czekam 5 sekund...
Czas minął!
Czekam 2 sekund...
Czas minął tak szybko!


TypeError: sleep.<locals>.inner() missing 1 required positional argument: 'func'

In [66]:
# dokonajmy więc zmian - wraps został dodany, aby niejako było to rozwiązanie kompletne (typ zwracanej funkcji oraz docstring).
import functools


# stosowana jest tu pewna sztuczka w sygnaturze funkcji
def sleep(_func=None, *, sleep_time=3):
    def inner(func):
        # @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # print(sleep_time)
            # sleep_time = sleep_time if isinstance(sleep_time, int) else 3
            print(f'Czekam {sleep_time} sekund...')
            time.sleep(sleep_time)
            return func(*args, **kwargs)
        return wrapper
    if _func is None:
        return inner
    else:
        return inner(_func)
    
@sleep(sleep_time=5)
def wakeup():
    print("Czas minął!")

@sleep(sleep_time=2)
def wakeup_fast():
    print("Czas minął tak szybko!")

# bez argumentów nie zadziała w tym przypadku!
@sleep
def wakeup_how():
    print("Czas minął dość szybko!")

wakeup()
wakeup_fast()
wakeup_how()

Czekam 5 sekund...
Czas minął!
Czekam 2 sekund...
Czas minął tak szybko!
Czekam 3 sekund...
Czas minął dość szybko!


Jak widać dodana została kolejna funkcja opakowująca. Zobaczmy wywołanie bez `@` co pozwoli lepiej zrozumieć dlaczego było to potrzebne.

In [78]:
def wakeup(name='Jan'):
    return f"Czas minął {name}!"

display(sleep)
display(sleep(sleep_time=5))
display(sleep(sleep_time=5)(wakeup))
display(sleep(sleep_time=5)(wakeup)())

# przekazanie funkcji wakeup do dekoratora bez podania wartości argumentu sleep_time
# nie dekorujemy żadnej funkcji
display(sleep()) 

# zwraca funkcję wakeup jeżeli użyjemy @functools.wraps albo wrapper bez użycia tego dekoratora
display(sleep(wakeup)) 

# wywołuje wrapper, który jak wiemy wywołuje func() czyli przekazaną funkcję wakeup
display(sleep(wakeup)()) 

# przekazuje do sleep wynik wywołania funkcji wakeup('Adam'), ale nie jest ona wywoływana
display(sleep(wakeup('Adam'))) 

# tu będzie błąd, gdyż do dekoratora trafi str (jak powyżej), ale zostanie on wywołany
# ang. call a to nie jest dopuszczalna operacja (np. 'Czas minął Adam!'())
# display(sleep(wakeup('Adam'))()) 

# wywołamy sleep z domyślnym parametrem sleep_time, które jak widać w wywołaniu sleep() powyżej zwróci funkcję inner
# co oznacza, że to wywołanie to faktycznie wywołanie inner(wakeup), które zwróci wrapper (ale jej nie wywołujemy)
display(sleep()(wakeup))

# jeżeli teraz wywołamy tę zwracaną funkcję
display(sleep()(wakeup)())

# to teraz spróbujmy przekazać argument do wakeup tak, aby to zadziałało
display(sleep()(wakeup)('Adam'))

display(sleep(sleep_time=6)(wakeup)('Antonio'))

<function __main__.sleep(_func=None, *, sleep_time=3)>

<function __main__.sleep.<locals>.inner(func)>

<function __main__.sleep.<locals>.inner.<locals>.wrapper(*args, **kwargs)>

Czekam 5 sekund...


'Czas minął Jan!'

...

'Czas minął Jan!'

Czekam 3 sekund...


'Czas minął Adam!'

Czekam 6 sekund...


'Czas minął Antonio!'

W Pythonie (i innych językach programowania zorientowanych obiektowo) możemy również wykorzystać dekoratory klas, które są również traktowane jako jeden z wzorców projektowych. Tutaj cała sztuka polega na przesłonięciu metod `__init__` oraz `__call__` w klasie dekoratora.

In [37]:
class TimerDecorator:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        start_time = time.time()
        result = self.func(*args, **kwargs)
        end_time = time.time()
        print(f"Funkcja {self.func.__name__} wykonała się w czasie {end_time - start_time:.08f} sekund")
        return result


@TimerDecorator
def powtorz(co, ile):
    return co * ile


powtorz('Foo ', 20)

Funkcja powtorz wykonała się w czasie 0.00000143 sekund


'Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo Foo '

## Zadania

1. Stwórz dekorator bezargumentowy funkcji, który będzie logował (log w rozumieniu dziennik zdarzeń) zdarzenie wywołania funkcji udekorowanej. Postać łańcucha znaków, który zwraca (niech tu będzie po prostu print) jest następujący:
[data i czas wywołania] nazwa funkcji argumenty jej wywołania (nazwa i wartość jeżeli to możliwe - można pomyśleć o zwróceniu postaci łańcuchowej *args oraz **kwargs przekazanych do tej funkcji).

2. Napisz dekorator `require_permission`, który będzie możliwy do użycia biorąc pod uwagę poniższy scenariusz jego użycia:

```python
class User:
    def __init__(self, permissions):
        self.permissions = permissions

    def has_permission(self, permission):
        return permission in self.permissions

@require_permission('admin')
def delete_user(user, user_id):
    print(f"User {user_id} deleted")

# Przykładowe wywołanie
admin_user = User(permissions=['admin'])
delete_user(admin_user, 123)
```

3. Stwórz dekorator klasy o nazwie `Singleton` i zaimplementuj logikę wzorca singleton dla udekorowanej własnej klasy. Przetestuj jego działanie.